## VGG 块的结构

VGG 块由 N 个卷积层构成


这 N 个卷积层中间没有池化层

只在最后有一个最大池化层

其中第一次卷积运算会改变输入数据的通道，后面的卷积运算不再改变通道数

激活函数使用 ReLU 函数

<br>

## VGG 块的示例

In [1]:
from torch import nn

'''
conv_num      -- 卷积层的个数
in_channels   -- 输入数据的通道数
out_channels  -- 输出数据的通道数（只在第一次卷积运算时改变输入数据的通道数）
'''
def VGG_BLOCK(conv_num, in_channels, out_channels):
    layers = []
    for _ in range(conv_num):
        layers.append(nn.Conv2d(in_channels, out_channels, 3, 1, 1))
        layers.append(nn.ReLU())
        in_channels = out_channels   # 后面的卷积运算不再改变输入数据的通道数
    
    # 最后添加一个最大池化层
    layers.append(nn.MaxPool2d(2, 2, 0))

    return layers

<br>

## VGG 的结构

VGG 由多个 VGG 块、2 个隐藏层、1 个输出层构成

两个隐藏层后面都添加一个 Dropout 层来防止模型过拟合

激活函数均使用 ReLU 函数

<br>

## VGG 的一个示例

In [2]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.Sequential(
            # 5 个 VGG 块
            *VGG_BLOCK(1, 1, 64),
            *VGG_BLOCK(1, 64, 128),
            *VGG_BLOCK(2, 128, 256),
            *VGG_BLOCK(2, 256, 512),
            *VGG_BLOCK(2, 512, 512),

            nn.Flatten(),

            # 第一个隐藏层，尾部衔接 Dropout 层
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            # 第二个隐藏层，尾部衔接 Dropout 层
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            # 输出层
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.layers(x)
        return x